## Der Zeit
#### Code for scraping Der Zeit, using their API
- https://www.zeit.de/index?utm_referrer=https%3A%2F%2Fwww.google.com

In [10]:
#Get URL's

import requests
import pandas as pd 
import urllib
from bs4 import BeautifulSoup
import nltk

# settings for the general site
# using my api key from a .txt document
fname = '/Users/emilymartin/Documents/Data_Science/Fluechtlingskrise-Sentiment-Analysis/my_token.txt'
f = open(fname)
my_token = f.read().strip()
f.close()
base_url = 'http://api.zeit.de'
# settings for the specific endpoint
endpoint = 'content'
endpoint_url = f'{base_url}/{endpoint}?api_key={my_token}'
#headers = {'X-Authorization': my_token}

# settings for the specific search query
keyword_search = 'Migranten+Fluechtling AND release_date:[2015-01-01T00:00:00Z TO 2015-12-31T23:59:59.999Z]' # search for all occurrences of 'Migrant' from 2015
limit = 1000 # Lets get as many as possible


# pass the settings and parameters to the get request
search_params = {'q':keyword_search, 'limit':limit} # see Zeit site for additional options http://developer.zeit.de/docs/content/
response = requests.get(endpoint_url, params=search_params) # send the request!

# look at the keys for the json dictionary produced by the pull request
print(response.json().keys())

# the data itself is in 'matches'. Select the matches element from the dictionary and convert it to a dataframe:
df = pd.DataFrame(response.json()['matches'])

#print(f'Your options are: {df.columns}')
#print(df[['title','href','release_date']])


df.head()

dict_keys(['matches', 'found', 'limit', 'offset'])


,subtitle,uuid,title,href,release_date,uri,snippet,supertitle,teaser_text,teaser_title
0,Ein deutscher Führerschein ist für viele Flüch...,7ykdw6fWIZkfEoCA5HrHKJ,Mahmood im Schilderwald,http://www.zeit.de/2015/51/fuehrerschein-fluec...,2015-12-31T02:51:37Z,http://api.zeit.de/content/7ykdw6fWIZkfEoCA5HrHKJ,". ""Wer Auto fahren kann, hat beruflich viel me...",Führerschein für Flüchtlinge,Ein deutscher Führerschein ist für viele Flüch...,Mahmood im Schilderwald
1,Atticus Lish erzählt souverän von der unwahrsc...,4wLrvdk6M6AKGig80RgjfP,Zwei zähe Einzelgänger,http://www.zeit.de/2015/51/vorbereitung-auf-da...,2015-12-31T01:56:01Z,http://api.zeit.de/content/4wLrvdk6M6AKGig80RgjfP,Kriegserlebnisse sich an die Oberfläche arbei...,Atticus Lish,Atticus Lish erzählt souverän von der unwahrsc...,Zwei zähe Einzelgänger
2,"Von Container-Millionären, Pegida-Anführerinne...",1t6ENxcNNbG830MqMoPiTB,Fortsetzung folgt – jetzt,http://www.zeit.de/2016/01/geschichten-2015-fo...,2015-12-30T09:00:08Z,http://api.zeit.de/content/1t6ENxcNNbG830MqMoPiTB,Bachmann hatten sich auseinandergelebt. Sie s...,Geschichten 2015,"Von Container-Millionären, Pegida-Anführerinne...",Fortsetzung folgt – jetzt
3,Nach der Grenzschließung in Ungarn führt die F...,5a6YZ9hEdbbfjhc2YQXnDP,Anhaltend hohe Flüchtlingszahlen auf Balkanroute,http://www.zeit.de/gesellschaft/2015-12/slowen...,2015-12-29T22:14:02Z,http://api.zeit.de/content/5a6YZ9hEdbbfjhc2YQXnDP,Auch zum Jahresende kommen weiter täglich Taus...,Slowenien,Nach der Grenzschließung in Ungarn führt die F...,Anhaltend hohe Flüchtlingszahlen auf Balkanroute
4,Integrationspflicht für Flüchtlinge? In der De...,1EN443vyRBFWFwyCqHQbTJ,Laut Özoğuz schürt Union Vorurteile gegen Flüc...,http://www.zeit.de/politik/deutschland/2015-12...,2015-12-29T08:24:55Z,http://api.zeit.de/content/1EN443vyRBFWFwyCqHQbTJ,Opposition und Koalitionspartner kritisieren d...,Integration,Integrationspflicht für Flüchtlinge? In der De...,Laut Özoğuz schürt Union Vorurteile gegen Flüc...


In [2]:
#Get all the URL's into a list
url_lst = df.href.tolist()
# Looks like 573 is what we got
len(url_lst)

573

In [14]:

art_dict = {} #Initiate empty dictionary 
x = ' ' #For joining the text we are collecting
for u in url_lst:                 #Loop through each url and use Beautiful Soup to get the parts we need
    response = requests.get(u)
    if response.status_code=='404': #in case it runs into any 404 messages
        print(url)
        pass
    soup = BeautifulSoup(response.text, 'html.parser')
    t = soup.find("div", {"class":"article-body"})
    if t==None:
        art_dict[u] = 'None' # A few links are broken it seems
        pass
        #print(t)
    else:
        paras = t.find_all("p")           #Within this div all p's hold the article text
        a = [p.text for p in paras]           #Nifty list comprehension to turn the bs4 object into text
        text = x.join(a)      #Join text
        art_dict[u] = text                #Add to dictionary with keys as URL's and values as text

#art_dict

In [16]:
#Map the text to a new column in our dataframe using the href keys, which are also a column in df. How cool!
df['text']= df['href'].map(art_dict)
#Get the word counts
word_c = df.text.str.split().map(len)
df['word_count'] = word_c

# I am going to reduce the number of columns so it is more similar to my other df's, but there are still here
df = df[['title', 'href', 'text', 'release_date', 'word_count']]

In [17]:
# Adding more columns
import warnings; warnings.simplefilter('ignore')
df['sent_count'] = df['text'].map(lambda s: len(nltk.sent_tokenize(s))) 
df['toks'] = df['text'].map(lambda t: len(nltk.word_tokenize(t))) 
df['types'] = df['text'].map(lambda x: len(set(nltk.word_tokenize(x)))) 
df['TTR'] = df.types/df.toks
df.head()

,title,href,text,release_date,word_count,sent_count,toks,types,TTR
0,Mahmood im Schilderwald,http://www.zeit.de/2015/51/fuehrerschein-fluec...,Als er vor über zehn Jahren Autofahren gelernt...,2015-12-31T02:51:37Z,1175,99,1390,608,0.437410
1,Zwei zähe Einzelgänger,http://www.zeit.de/2015/51/vorbereitung-auf-da...,"Wo Zou Lei herkommt, ist das Leben nicht leich...",2015-12-31T01:56:01Z,1155,75,1363,686,0.503302
2,Fortsetzung folgt – jetzt,http://www.zeit.de/2016/01/geschichten-2015-fo...,"Lok Leipzig ist ratlos, was aus Mario Basler w...",2015-12-30T09:00:08Z,313,26,372,229,0.615591
3,Anhaltend hohe Flüchtlingszahlen auf Balkanroute,http://www.zeit.de/gesellschaft/2015-12/slowen...,Auch zum Jahresende kommen weiter täglich Taus...,2015-12-29T22:14:02Z,362,23,402,236,0.587065
4,Laut Özoğuz schürt Union Vorurteile gegen Flüc...,http://www.zeit.de/politik/deutschland/2015-12...,Opposition und Koalitionspartner kritisieren d...,2015-12-29T08:24:55Z,379,24,447,237,0.530201


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         573 non-null    object 
 1   href          573 non-null    object 
 2   text          573 non-null    object 
 3   release_date  573 non-null    object 
 4   word_count    573 non-null    int64  
 5   sent_count    573 non-null    int64  
 6   toks          573 non-null    int64  
 7   types         573 non-null    int64  
 8   TTR           573 non-null    float64
dtypes: float64(1), int64(4), object(4)
memory usage: 40.4+ KB


In [19]:
# Pickle
pd.to_pickle(df, "zeit_df.pkl")